In [ ]:
import cv2
import numpy as np
import psutil
import time
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
import matplotlib.pyplot as plt

def gaussian_filter(image, ksize=5, sigma=1.0):
    """
    Apply Gaussian Filter on the input image using CPU.

    Parameters:
    - image: Input image (binary or 3-channel)
    - ksize: Kernel size (odd integer)
    - sigma: Gaussian kernel standard deviation

    Returns:
    - filtered_image: Output image after applying Gaussian filter
    """
    return cv2.GaussianBlur(image, (ksize, ksize), sigma)

def measure_metrics(image, filtered_image, inference_time):

    cpu_usage = psutil.cpu_percent(interval=None)

    process = psutil.Process()
    ram_usage = process.memory_info().rss / (1024 * 1024)

    psnr_value = psnr(image, filtered_image)

    metrics = {
        "CPU Usage (%)": cpu_usage,
        "RAM Usage (MB)": ram_usage,
        "Inference Time (ms)": inference_time,
        "PSNR (dB)": psnr_value,
    }

    return metrics

def process_image(image_path, output_path, ksize=5, sigma=1.0):
    """
    Load image, apply Gaussian Filter, measure metrics, and save the result.

    Parameters:
    - image_path: Path to the input image
    - output_path: Path to save the denoised image
    - ksize: Kernel size (odd integer)
    - sigma: Gaussian kernel standard deviation
    """
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    if image.shape[0] > 720 or image.shape[1] > 1280:
        raise ValueError("Image size exceeds the allowed dimensions of 1280x720p")

    start_time = time.time()

    filtered_image = gaussian_filter(image, ksize=ksize, sigma=sigma)

    inference_time = (time.time() - start_time) * 1000

    metrics = measure_metrics(image, filtered_image, inference_time)

    cv2.imwrite(output_path, filtered_image)

    return metrics

def process_dataset(root_folder, ksize=5, sigma=1.0):
    """
    Process all images in the dataset and save denoised images to the respective folders.

    Parameters:
    - root_folder: Path to the root dataset folder
    - ksize: Kernel size (odd integer)
    - sigma: Gaussian kernel standard deviation
    """
    noisy_folders = ['noisy5', 'noisy15', 'noisy35']
    denoised_folders = ['denoised5', 'denoised15', 'denoised35']

    total_cpu_usage = 0
    total_ram_usage = 0
    total_inference_time = 0
    total_psnr = 0
    image_count = 0

    for noisy_folder, denoised_folder in zip(noisy_folders, denoised_folders):
        input_folder = os.path.join(root_folder, noisy_folder)
        output_folder = os.path.join(root_folder, denoised_folder)

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for filename in os.listdir(input_folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(input_folder, filename)
                output_path = os.path.join(output_folder, filename)

                metrics = process_image(image_path, output_path, ksize, sigma)

                total_cpu_usage += metrics["CPU Usage (%)"]
                total_ram_usage += metrics["RAM Usage (MB)"]
                total_inference_time += metrics["Inference Time (ms)"]
                total_psnr += metrics["PSNR (dB)"]
                image_count += 1

    if image_count > 0:
        avg_cpu_usage = total_cpu_usage / image_count
        avg_ram_usage = total_ram_usage / image_count
        avg_inference_time = total_inference_time / image_count
        avg_psnr = total_psnr / image_count

        print(f"Total CPU Usage (%): {avg_cpu_usage:.2f}")
        print(f"Total RAM Usage (MB): {avg_ram_usage:.2f}")
        print(f"Total Inference Time (ms): {avg_inference_time:.2f}")
        print(f"Average PSNR (dB): {avg_psnr:.2f}")
    else:
        print("No images processed.")

if __name__ == "__main__":
    root_folder = '/content/drive/MyDrive/dataset'
    process_dataset(root_folder, ksize=5, sigma=1.0)
